<a href="https://colab.research.google.com/github/syedahafsa12/Langgraph_Business_Agent/blob/main/Business_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BUSINESS LANGRAPH AGENT

In [ ]:
!pip install langgraph beautifulsoup4 requests
!pip install streamlit cloudflared

!pip install -q langchain_google_genai langchain_core langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.2/137.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.8 MB/s eta 0:00:00
  Created wheel for cloudflared: filename=cloudflared-1.0.0.2-py3-none-any.whl size=2984 sha256=b78e958c6e500cc64e77f995ca34d63f1fd3dac8a46c749cbb7b2510ccd106af
  Stored in directory: /root/.cache/pip/wheels/6b/00/71/ea5fa0bfe899be948e296b3f357de632b8c30b6546c1e50100
Successfully built cloudflared
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import requests
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

# -------------------------------
# 📌 API CONFIGURATION
# -------------------------------
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
SEARCH_ENGINE_ID = userdata.get("SEARCH_ENGINE_ID")
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")

if not GOOGLE_API_KEY or not SEARCH_ENGINE_ID or not GEMINI_API_KEY:
    raise ValueError("❌ Missing API keys. Please set GOOGLE_API_KEY, SEARCH_ENGINE_ID, and GEMINI_API_KEY.")

# Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=GEMINI_API_KEY)

# Chat memory for the last 10 chats
chat_memory = []

# -------------------------------
# 📌 HELPER FUNCTIONS
# -------------------------------

def extract_gender_and_style(user_query: str, state: dict):
    """Extract gender and ask style for outfit queries when needed."""
    if "gender" not in state["user_info"] or state["user_info"]["gender"] == "unknown":
        if "outfit" in user_query.lower() or "dress" in user_query.lower():
            gemini_prompt = f"Analyze the input '{user_query}' and determine the gender (male, female) if mentioned. Respond with 'male', 'female', or 'unknown'."
            gemini_response = llm.invoke([{"role": "user", "content": gemini_prompt}])
            extracted_gender = gemini_response.content.strip().lower()
            if extracted_gender not in ["male", "female"]:
                extracted_gender = input("👤 What is the gender for this outfit query? (male/female): ").strip().lower()
            state["user_info"]["gender"] = extracted_gender

    if "style" not in state["user_info"] or not state["user_info"]["style"]:
        if "outfit" in user_query.lower() or "dress" in user_query.lower():
            style_choice = input("👤 Traditional or modern attire? ").strip().lower()
            state["user_info"]["style"] = style_choice

def fetch_images_from_pinterest(query: str, num_results: int = 3):
    """Fetch relevant Pinterest images."""
    url = (
        f"https://www.googleapis.com/customsearch/v1?key={GOOGLE_API_KEY}&cx={SEARCH_ENGINE_ID}&q={query} site:pinterest.com&searchType=image&num={num_results}"
    )
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return [item["link"] for item in data.get("items", []) if "pinterest.com/pin/" in item["link"]]
    except Exception as e:
        print(f"❌ Error fetching Pinterest images: {e}")
        return []

def fetch_search_results(query: str, num_results: int = 5):
    """Fetch web search results."""
    url = f"https://www.googleapis.com/customsearch/v1?key={GOOGLE_API_KEY}&cx={SEARCH_ENGINE_ID}&q={query}&num={num_results}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return [
            {"title": item["title"], "link": item["link"]}
            for item in data.get("items", [])
        ]
    except Exception as e:
        print(f"❌ Error fetching web results: {e}")
        return []

def clarify_user_intent(user_query: str, state: dict):
    """Clarify if the user wants to buy an outfit or see ideas."""
    if "outfit" in user_query.lower() or "dress" in user_query.lower():
        if "intent" not in state:
            user_choice = input("👗 Would you like to buy an outfit or see outfit ideas? (buy/ideas): ").strip().lower()
            if user_choice in ["buy", "ideas"]:
                state["intent"] = user_choice
            else:
                print("❓ Please specify either 'buy' or 'ideas'.")
                return clarify_user_intent(user_query, state)  # Retry
        return state["intent"]
    return None

def generate_response(user_query: str, user_info: dict):
    """Generate a concise and joyful response using Gemini."""
    gemini_prompt = f"Create a concise, joyful response for the query '{user_query}' with gender '{user_info.get('gender', 'unknown')}' and style '{user_info.get('style', 'unknown')}'."
    gemini_response = llm.invoke([{"role": "user", "content": gemini_prompt}])
    return gemini_response.content.strip()

def display_results(response: dict):
    """Display results based on the response."""
    print(f"\n🤖 Gemini says: '{response['joyful_message']}'")
    if response.get("images"):
        print("\n📸 Related Images:")
        for idx, img in enumerate(response["images"], start=1):
            print(f"{idx}. {img}")
    if response.get("search_results"):
        print("\n🔍 Search Results:")
        for idx, result in enumerate(response["search_results"], start=1):
            print(f"{idx}. {result['title']}: {result['link']}")

def save_to_memory(user_query, response):
    """Save the last 10 chats to memory."""
    global chat_memory
    chat_memory.append({"user_query": user_query, "response": response})
    if len(chat_memory) > 10:
        chat_memory.pop(0)

def recall_memory(query: str):
    """Recall memory based on the user's query."""
    global chat_memory
    if "name" in query.lower():
        for chat in reversed(chat_memory):
            if "i am" in chat["user_query"].lower():
                name = chat["user_query"].split("i am", 1)[1].strip()
                return f"Your name is {name}!"
    return "I don't recall anything specific about that."

# -------------------------------
# 📌 GRAPH NODES
# -------------------------------

def handle_user_query(state):
    """Analyze user query, infer missing details, and respond dynamically."""
    user_query = state["user_query"]

    # Step 1: Extract gender and style if needed
    extract_gender_and_style(user_query, state)

    # Step 2: Clarify intent for outfit queries
    user_intent = clarify_user_intent(user_query, state)
    if user_intent == "ideas":
        user_query += f" site:pinterest.com {state['user_info']['style']}"
    elif user_intent == "buy":
        user_query += " Pakistani outfits buy"

    # Step 3: Use Gemini to determine query type
    gemini_query_type_prompt = f"Analyze the input '{user_query}'. Respond with one word: 'direct', 'search', or 'images'."
    query_type = llm.invoke([{"role": "user", "content": gemini_query_type_prompt}]).content.strip().lower()

    # Case 1: Direct query
    if query_type == "direct":
        joyful_message = generate_response(user_query, state["user_info"])
        response = {"joyful_message": joyful_message, "images": [], "search_results": []}

    # Case 2: Search query
    elif query_type == "search":
        search_results = fetch_search_results(user_query)
        joyful_message = generate_response(user_query, state["user_info"])
        response = {"joyful_message": joyful_message, "images": [], "search_results": search_results}

    # Case 3: Images query restricted to Pinterest
    elif query_type == "images":
        images = fetch_images_from_pinterest(user_query)
        joyful_message = generate_response(user_query, state["user_info"])
        response = {"joyful_message": joyful_message, "images": images[:1], "search_results": []}  # Limit to one image

    # Default fallback
    else:
        response = {"joyful_message": "I'm not sure how to help with that. Could you clarify?", "images": [], "search_results": []}

    save_to_memory(user_query, response)  # Save chat to memory
    state["response"] = response
    return state

# -------------------------------
# 📌 GRAPH DEFINITION
# -------------------------------
builder = StateGraph(dict)
builder.add_node("handle_user_query", handle_user_query)
builder.add_edge(START, "handle_user_query")
builder.add_edge("handle_user_query", END)
graph = builder.compile()

# -------------------------------
# 📌 MAIN PROGRAM
# -------------------------------
if __name__ == "__main__":
    print("👗 Welcome to the Fashion AI Agent! Ask me anything about fashion or trends!")
    while True:
        state = {"user_query": "", "user_info": {"gender": "unknown", "style": ""}, "response": {}}
        user_query = input("💬 What are you looking for? (e.g., 'Eid dresses for women' or 'Pakistani brand sales'): ")
        if user_query.lower() in ["what did i tell you earlier?", "recall memory"]:
            print(recall_memory(user_query))
            continue
        state["user_query"] = user_query
        state = graph.invoke(state)
        display_results(state["response"])


👗 Welcome to the Fashion AI Agent! Ask me anything about fashion or trends!

🤖 Gemini says: 'Pinterest is bursting with gorgeous traditional Eid dresses for girls!  Prepare to be inspired! ✨'

🔍 Search Results:
1. Pin page: https://www.pinterest.com/pin/creative-photography--33143747252211318/
2. Latest Eid Dress Designs 2023: https://www.pinterest.com/pin/latest-eid-dress-designs-2023--627548529350579739/
3. Chotibuti Embroidered Lehenga Set | Ivory, Thread Work, Mulmul ...: https://www.pinterest.com/pin/chotibuti-embroidered-lehenga-set--799670477604921372/
4. African Girl Eid Style: https://www.pinterest.com/pin/african-girl-eid-style--54113633017752563/
5. Traditional Dress Hide Face: https://www.pinterest.com/ideas/traditional-dress-hide-face/928914052721/
